# (Run this notebook locally)

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import tqdm

In [2]:
# Import keras
import keras

Using TensorFlow backend.


In [3]:
# Graph
from py2neo import Graph

# Input

In [7]:
PATH_TFMDATA = "/Users/david/TFM_DATA"
PATH_DATA = "/Users/david/TFM_DATA/spec"
data_files = os.listdir(PATH_DATA)

In [8]:
# PATH triplets
PATH_TRIPLETS = os.path.join(PATH_TFMDATA, "triplets.csv")
df = pd.read_csv(PATH_TRIPLETS, delimiter= ";")

In [9]:
# List
triplets_input = list(df["output"])
size_triplets_input = len(triplets_input)

# Sample
triplets_input_v1 = triplets_input[:1000]
size_triplets_input = len(triplets_input_v1)

# Artists
artists_labels = np.sort(df.a1.unique())
df_artists = pd.DataFrame(data = artists_labels, columns = ["artist"]).reset_index()
df_artists.columns = ["id","artist"]

# DF Artists with artists as index
df_artists_index = df_artists.set_index("artist")
num_artists = df_artists.shape[0]

In [11]:
graph = Graph(bolt = True, host = "localhost", name = "Spotify", user = "neo4j", password = "qrks")

## Catalan music

In [ ]:
name_file = "dfneo_CAT.csv"

In [ ]:
query = """
        MATCH (g:Genre)-[:GEN_ART]->(a:Artist)
        MATCH (a:Artist)-[:REL_ART]->(a2:Artist)
        WHERE toLower(g.genre_id) =~ '.*catala.*'
        RETURN a.artist_id, a2.artist_id

"""

In [ ]:
cursor = graph.run(query)
dfneo = pd.DataFrame.from_records(cursor, columns=cursor.keys())

In [ ]:
dfneo.to_csv(name_file, sep = ";", index = False)

## Popular music

In [41]:
name_file = "popular.csv"

In [19]:
query = """
        MATCH (n:Artist)-[:ART_TR]-(t:Track) 
        WHERE t.yt_views > 100000000  
        RETURN n.artist_id, n.artist_name
        ORDER BY t.yt_views DESC  LIMIT 100

"""

In [20]:
cursor = graph.run(query)
dfneo = pd.DataFrame.from_records(cursor, columns=cursor.keys())

In [25]:
selected_popular = [
    'Daddy Yankee','Ed Sheeran','Bruno Mars','Justin Bieber','Maroon 5','Katy Perry','Taylor Swift',
    'J Balvin', 'Passenger', 'Beyoncé', 'Adele', 'Maluma','Shakira', 'The Chainsmokers', 'Meghan Trainor',
    'Clean Bandit', 'Rihanna','Sean Paul', 'Ozuna', 'Sia', 'Dua Lipa', 'Coldplay', 'Ariana Grande','Nicki Minaj'
]
set_selected = set(selected_popular)

In [37]:
dfpopular = dfneo[dfneo["n.artist_name"].isin(set_selected)].copy()

In [38]:
dfpopular.reset_index(inplace=True)
dfpopular.drop("index", axis = 1, inplace = True)
dfpopular.columns = ["artist","name"]

In [42]:
dfpopular.to_csv(name_file, sep = ";", index = False)